# Load the pretrained model

Load annotations without Images

In [ ]:
curl -X GET http://url:8080/api/projects/1/export?exportType=CSV -H 'Authorization: Token xxx' --output 'annotations.csv' 

Archive Images (on Server!)

In [ ]:
tar -czvf /home/alex/all_imgs_11_10_23.tar.gz

Download archive from server and extract it

In [ ]:
scp alex@foodsnapai.com:/home/alex/all_imgs_11_10_23.tar.gz allImgs.tar.gz
tar -xzvf allImgs.tar.gz -C /home/alex/allImgs_extracted/

Convert annotations and images and preprocess them

In [ ]:
%run 'convert_csv_to_mlflow_csv.py' annotations.csv

In [ ]:
%run 'convert_pascal_to_googlecsv.py' --merged_csv annotations_mlflow.csv

Now rename the file paths in the file to their actual location (find & replace)

In [7]:
!pwd
%run 'png_to_jpeg.py' annotations_mlflow_shuffled.csv

/home/alex/tflite_model_maker_wsl2
convert png image files to jpeg imgs


0it [00:00, ?it/s]

Converted 22
Ignored 6206 files because: Not a PNG file
Ignored 0 files because: Not found


In [ ]:

%run preproc_imgs.py /home/alex/allImgs_extracted /home/alex/allImgs_extracted_smaller 800

In [8]:

%run normalize_csv.py annotations_mlflow_shuffled.csv annotations_mlflow_shuffled_n.csv

check the stats of the dataset splits

In [22]:
%run get_label_stats.py annotations_mlflow_shuffled_n.csv

There are 964 different files in this set
There are 14 different classes in this set
Classes: Apple, other, Egg, Cucumber, Onion, Bell-Pepper, Banana, Lemon, Tomato, Garlic, Carrot, Potato, Zucchini, Pumpkin
Apple occurs in 98 different files
other occurs in 874 different files
Egg occurs in 131 different files
Cucumber occurs in 255 different files
Onion occurs in 163 different files
Bell-Pepper occurs in 183 different files
Banana occurs in 134 different files
Lemon occurs in 103 different files
Tomato occurs in 259 different files
Garlic occurs in 115 different files
Carrot occurs in 146 different files
Potato occurs in 105 different files
Zucchini occurs in 119 different files
Pumpkin occurs in 43 different files

Class Distribution in Splits:

TRAIN:
Apple: 76 files (3.47%)
other: 696 files (31.78%)
Egg: 107 files (4.89%)
Cucumber: 202 files (9.22%)
Onion: 134 files (6.12%)
Bell-Pepper: 143 files (6.53%)
Banana: 105 files (4.79%)
Lemon: 88 files (4.02%)
Tomato: 206 files (9.41%)
G

Now you can start to load the model and train!

In [9]:
import numpy as np
import os

from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

spec = model_spec.get('efficientdet_lite0')

# Load the dataset

In [14]:
import os
current_directory = os.getcwd()
print("Current Working Directory:", current_directory)
file_path = '/home/alex/tflite_model_maker_wsl2/annotations_mlflow_shuffled_n.csv'

if os.path.exists(file_path):
    print(f"The file '{file_path}' exists.")
train_data, validation_data, test_data = object_detector.DataLoader.from_csv(file_path)

Current Working Directory: /home/alex/tflite_model_maker_wsl2
The file '/home/alex/tflite_model_maker_wsl2/annotations_mlflow_shuffled_n.csv' exists.


# Train the model

In [23]:
model = object_detector.create(train_data, model_spec=spec, batch_size=8, train_whole_model=True, validation_data=validation_data, epochs=50)

Epoch 1/50
96/96 [==============================] - 183s 2s/step - det_loss: 1.6233 - cls_loss: 1.0471 - box_loss: 0.0115 - reg_l2_loss: 0.0651 - loss: 1.6884 - learning_rate: 0.0090 - gradient_norm: 1.0933 - val_det_loss: 1.4922 - val_cls_loss: 0.9598 - val_box_loss: 0.0106 - val_reg_l2_loss: 0.0651 - val_loss: 1.5573
Epoch 2/50
96/96 [==============================] - 68s 704ms/step - det_loss: 1.3207 - cls_loss: 0.8493 - box_loss: 0.0094 - reg_l2_loss: 0.0651 - loss: 1.3858 - learning_rate: 0.0100 - gradient_norm: 1.5115 - val_det_loss: 1.3459 - val_cls_loss: 0.8727 - val_box_loss: 0.0095 - val_reg_l2_loss: 0.0651 - val_loss: 1.4110
Epoch 3/50
96/96 [==============================] - 32s 329ms/step - det_loss: 1.2075 - cls_loss: 0.7887 - box_loss: 0.0084 - reg_l2_loss: 0.0651 - loss: 1.2726 - learning_rate: 0.0099 - gradient_norm: 1.6517 - val_det_loss: 1.2879 - val_cls_loss: 0.8298 - val_box_loss: 0.0092 - val_reg_l2_loss: 0.0652 - val_loss: 1.3531
Epoch 4/50
96/96 [===============

Run on test data

In [27]:
model.evaluate(test_data)

2/2 [==============================] - 21s 7s/step



{'AP': 0.206375,
 'AP50': 0.28514978,
 'AP75': 0.24008648,
 'APs': 0.0,
 'APm': 0.1387649,
 'APl': 0.21842723,
 'ARmax1': 0.16211015,
 'ARmax10': 0.32523456,
 'ARmax100': 0.33753207,
 'ARs': 0.0,
 'ARm': 0.19437197,
 'ARl': 0.37167054,
 'AP_/other': 0.3038995,
 'AP_/Apple': 0.023664827,
 'AP_/Egg': 0.624711,
 'AP_/Cucumber': 0.24533091,
 'AP_/Onion': 0.31286484,
 'AP_/Bell-Pepper': 0.239969,
 'AP_/Banana': 0.40257305,
 'AP_/Lemon': 0.0,
 'AP_/Tomato': 0.16286746,
 'AP_/Garlic': 0.13826358,
 'AP_/Carrot': 0.19271539,
 'AP_/Potato': 0.15178475,
 'AP_/Zucchini': 0.07176155,
 'AP_/Pumpkin': 0.018844167}

Export the model to tflite model

In [24]:
model.export(export_dir='models/new_model_900_e50')

2023-11-13 00:52:04.885225: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'resample_p7/PartitionedCall' has 1 outputs but the _output_shapes attribute specifies shapes for 3 outputs. Output shapes may be inaccurate.


Estimated count of arithmetic ops: 1.797 G  ops, equivalently 0.898 G  MACs


2023-11-13 00:52:14.790028: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2023-11-13 00:52:14.790063: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2023-11-13 00:52:14.793011: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpjkoegxl0
2023-11-13 00:52:14.842613: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2023-11-13 00:52:14.842646: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmpjkoegxl0
2023-11-13 00:52:15.014224: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2023-11-13 00:52:16.130482: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/tmpjkoegxl0
2023-11-13 00:52:16.587339: I tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: success: OK. Took 1794787

Estimated count of arithmetic ops: 1.797 G  ops, equivalently 0.898 G  MACs


fully_quantize: 0, inference_type: 6, input_inference_type: 3, output_inference_type: 0
2023-11-13 00:53:28.994994: I tensorflow/compiler/mlir/lite/flatbuffer_export.cc:1963] Estimated count of arithmetic ops: 1.797 G  ops, equivalently 0.898 G  MACs



Run inference script for visual output

In [30]:
%run do_inference.py --input_csv annotations_mlflow_shuffled_n.csv\
                       --model_url models/new_model_900_e50/model.tflite --output_dir output_inference

  0%|          | 0/96 [00:00<?, ?it/s]

  1%|          | 1/96 [00:01<02:17,  1.45s/it]

Prediction saved to output_inference/prediction_ac0ed1b0-d9cac84833465320_IMG_20230809_130228_HDR.jpg


  2%|▏         | 2/96 [00:02<02:16,  1.46s/it]

Prediction saved to output_inference/prediction_a651e069-4888f0ad0981b6e0_20230907_120546.jpg


...

Prediction saved to output_inference/prediction_817e8228-084fc54a9db713bb_IMG_20230719_131936_HDR.jpg


 99%|█████████▉| 95/96 [02:17<00:01,  1.44s/it]

Prediction saved to output_inference/prediction_4f03357a-96510c9d0eb73af7_20230815_073826.jpg


100%|██████████| 96/96 [02:19<00:00,  1.45s/it]

Prediction saved to output_inference/prediction_287807f6-85e6a1d3b1b5dfae_4919E65E-19C8-40C6-8099-AC9E4243B31E.jpeg
